# Homework 04
# Part 1 - Combine Sp21 Course Info with Requirements

## Prep

In [296]:
from bs4 import BeautifulSoup
import pandas as pd
import re

## 1. Read the 2021 course schedule into a DataFrame
### Create DataFrame

I read in the file, parse the html, and extract data from the html.
I begin by making lists for each kind of data I want to make sure I am extracting data for each column properly.

In [297]:
# read in file, parse html
f = open('s21schedule.html', 'r')
dom = BeautifulSoup(f, 'html.parser')
f.close()
# print(dom.prettify())

In [298]:
# extract data
selector = 'div.faculty ul.schedule-listing li.row div'
rows = dom.select(selector)

In [299]:
# make lists to check that data is being extracted properly, 
# ignore courses that were cancelled 
number_sections = []
names = []
instructors = []
times = []

for row in rows:
    spans = row.select("span:not(.il)")
    if spans == []:
        continue

    m = re.search("(CSCI\-U?G?A\.\d{4}\-\d{3}|MAINT\-GA\.\d{4}\-\d{3})", spans[0].text.replace('\u200b',''))
    num = m[0]
    
    name = spans[1].text.strip().replace('\n', '')
    name = ' '.join(name.split())
    
    inst = spans[2].text.strip().replace('Office Hours', '')
    inst = re.sub(r'\n+ +', '\n', inst).strip()
    inst = re.sub(r'\n+', ', ', inst).strip()
    if inst == 'CANCELLED':
        continue
    
    time = spans[3].text.strip()
    
    number_sections.append(num)
    names.append(name)
    instructors.append(inst)
    times.append(time)
    
#     print(num, name, inst, time)

In [300]:
# make sure elements of each list look as expected
# number_sections[:10]
# names[:10]
# instructors[:10]
# times[:10]

I now go through the html again to extract the data, saving each row in a list and appending to a list of lists which will become the data for my DataFrame.

In [301]:
course_data = []

for row in rows:
    spans = row.select("span:not(.il)")
    if spans == []:
        continue
    row_data = []
    
    m = re.search("(CSCI\-U?G?A\.\d{4}\-\d{3}|MAINT\-GA\.\d{4}\-\d{3})", spans[0].text.replace('\u200b',''))
    num = m[0]
    row_data.append(num)
    
    name = spans[1].text.strip().replace('\n', '')
    name = ' '.join(name.split())
    row_data.append(name)
    
    inst = spans[2].text.strip().replace('Office Hours', '')
    inst = re.sub(r'\n+ +', '\n', inst).strip()
    inst = re.sub(r'\n+', ', ', inst).strip()
    if inst == 'CANCELLED':
        continue
    row_data.append(inst)
    
    time = spans[3].text.strip()
    row_data.append(time)
    
    course_data.append(row_data)

In [302]:
# check that data is as expected
course_data[:5]

[['CSCI-GA.1144-001', 'PAC II*', 'Mohamed Zahran', 'T 6:00-8:30PM'],
 ['CSCI-GA.1144-002',
  'PAC II Recitation',
  'Gurkirat Singh Bajwa',
  'R 7:10-8:00PM'],
 ['CSCI-GA.1170-001',
  'Fundamental Algorithms',
  'Yevgeniy Dodis',
  'T 7:10-9:00PM'],
 ['CSCI-GA.1170-002',
  'Fundamental Algorithms Recitation',
  'Alex Bienstock, Charles Peyser, Fengyuan Liu',
  'R 8:10-9:00PM'],
 ['CSCI-GA.1170-003',
  'Fundamental Algorithms Recitation',
  'Harish Karthikeyan',
  'R 8:10-9:00PM']]

I create the DataFrame, using my list of lists from above as the data and a list of specified column names.

In [303]:
# create dataframe
cols = ['Number-Section', 'Name', 'Instructor', 'Time']
df = pd.DataFrame(course_data, columns=cols)
df

,Number-Section,Name,Instructor,Time
0,CSCI-GA.1144-001,PAC II*,Mohamed Zahran,T 6:00-8:30PM
1,CSCI-GA.1144-002,PAC II Recitation,Gurkirat Singh Bajwa,R 7:10-8:00PM
2,CSCI-GA.1170-001,Fundamental Algorithms,Yevgeniy Dodis,T 7:10-9:00PM
3,CSCI-GA.1170-002,Fundamental Algorithms Recitation,"Alex Bienstock, Charles Peyser, Fengyuan Liu",R 8:10-9:00PM
4,CSCI-GA.1170-003,Fundamental Algorithms Recitation,Harish Karthikeyan,R 8:10-9:00PM
...,...,...,...,...
145,CSCI-UA.0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 9:30-10:45AM
146,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,MW 3:30-4:45PM
147,CSCI-UA.0480-521,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 11:00-12:15PM
148,CSCI-UA.0480-522,Special Topics: Algorithmic Problem Solving,Kunal Khatri,W 3:30-4:45PM


### Break apart Number-Section into separate columns for Number and Section

I use regex to extract the course number and section from the Number-Section column and store the results in the appropriate new column.

In [304]:
result = df['Number-Section'].str.extract("(M?A?I?N?T?C?S?C?I?\-U?G?A\.\d{4})\-(\d{3})")
df['Number'] = result[0]
df['Section'] = result[1]
# pd.set_option('display.max_rows', 200)
pd.set_option('display.max_rows', 10)
df

,Number-Section,Name,Instructor,Time,Number,Section
0,CSCI-GA.1144-001,PAC II*,Mohamed Zahran,T 6:00-8:30PM,CSCI-GA.1144,001
1,CSCI-GA.1144-002,PAC II Recitation,Gurkirat Singh Bajwa,R 7:10-8:00PM,CSCI-GA.1144,002
2,CSCI-GA.1170-001,Fundamental Algorithms,Yevgeniy Dodis,T 7:10-9:00PM,CSCI-GA.1170,001
3,CSCI-GA.1170-002,Fundamental Algorithms Recitation,"Alex Bienstock, Charles Peyser, Fengyuan Liu",R 8:10-9:00PM,CSCI-GA.1170,002
4,CSCI-GA.1170-003,Fundamental Algorithms Recitation,Harish Karthikeyan,R 8:10-9:00PM,CSCI-GA.1170,003
...,...,...,...,...,...,...
145,CSCI-UA.0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 9:30-10:45AM,CSCI-UA.0480,057
146,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,MW 3:30-4:45PM,CSCI-UA.0480,069
147,CSCI-UA.0480-521,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 11:00-12:15PM,CSCI-UA.0480,521
148,CSCI-UA.0480-522,Special Topics: Algorithmic Problem Solving,Kunal Khatri,W 3:30-4:45PM,CSCI-UA.0480,522


### Show results

In [305]:
# info to show data types and counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Number-Section  150 non-null    object
 1   Name            150 non-null    object
 2   Instructor      150 non-null    object
 3   Time            150 non-null    object
 4   Number          150 non-null    object
 5   Section         150 non-null    object
dtypes: object(6)
memory usage: 7.2+ KB


In [306]:
# first 5 rows
df.head()

,Number-Section,Name,Instructor,Time,Number,Section
0,CSCI-GA.1144-001,PAC II*,Mohamed Zahran,T 6:00-8:30PM,CSCI-GA.1144,001
1,CSCI-GA.1144-002,PAC II Recitation,Gurkirat Singh Bajwa,R 7:10-8:00PM,CSCI-GA.1144,002
2,CSCI-GA.1170-001,Fundamental Algorithms,Yevgeniy Dodis,T 7:10-9:00PM,CSCI-GA.1170,001
3,CSCI-GA.1170-002,Fundamental Algorithms Recitation,"Alex Bienstock, Charles Peyser, Fengyuan Liu",R 8:10-9:00PM,CSCI-GA.1170,002
4,CSCI-GA.1170-003,Fundamental Algorithms Recitation,Harish Karthikeyan,R 8:10-9:00PM,CSCI-GA.1170,003


In [307]:
# last 5 rows
df.tail()

,Number-Section,Name,Instructor,Time,Number,Section
145,CSCI-UA.0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 9:30-10:45AM,CSCI-UA.0480,057
146,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,MW 3:30-4:45PM,CSCI-UA.0480,069
147,CSCI-UA.0480-521,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 11:00-12:15PM,CSCI-UA.0480,521
148,CSCI-UA.0480-522,Special Topics: Algorithmic Problem Solving,Kunal Khatri,W 3:30-4:45PM,CSCI-UA.0480,522
149,CSCI-UA.0480-523,Special Topics: Algorithmic Problem Solving,Samasth Ananda,W 3:30-4:45PM,CSCI-UA.0480,523


In [308]:
# random sampling of 5 rows
df.sample(5)

,Number-Section,Name,Instructor,Time,Number,Section
11,CSCI-GA.2130-001,Compiler Construction,Alexander Alekseyev,M 7:10-9:00PM,CSCI-GA.2130,001
6,CSCI-GA.2110-001,Programming Languages,Cory Plock,M 7:10-9:00PM,CSCI-GA.2110,001
31,CSCI-GA.2945-004,Adv Topics Num. Analysis: Stochastic Modeling ...,Benjamin Peherstorfer,R 1:25-3:15PM,CSCI-GA.2945,004
21,CSCI-GA.2520-001,Bioinformatics and Genomes,"Brian Parker, Richard Bonneau",F 1:00-3:45PM,CSCI-GA.2520,001
29,CSCI-GA.2945-002,Adv Topics Num. Analysis: High Performance Com...,Georg Stadler,M 1:25-3:15PM,CSCI-GA.2945,002


## 2. Read the 2021 course catalog into a DataFrame
### Create DataFrame

As done in (1), I read in the file, parse the html, and extract data from the html, making my initial lists to make sure I am extracting data for each column properly.

In [309]:
# read in file, parse html
f = open('catalog.html', 'r')
dom2 = BeautifulSoup(f, 'html.parser')
f.close()
# print(dom.prettify())

In [310]:
# extract data
selector = 'div.faculty div.row ul.courses-listing li.col-sm-12'
rows = dom2.select(selector)
# number, prereqs, points

In [329]:
# make lists to check that data is being extracted properly
numbers = []
prereqs = []
points = []

for row in rows:
    spans = row.select('p.bold')
    if spans == []:
        continue

    m1 = re.search("C?S?C?I?M?A?I?N?T?C?O?R?E?F?R?S?E?M?\-U?G?A\.\d{4}", spans[0].text.strip())
    if m1!=None:
        num = m1[0]
       
    m2 = re.search("Prerequisites: (.*)", spans[2].text.strip())
    if m2!=None:
        prq = m2[1]
    
    m3 = re.search("(.*) Points", spans[1].text.strip())
    if m3!=None:
        pts = m3[1]
    else:
        pts = None
    
    numbers.append(num)
    prereqs.append(prq)
    points.append(pts)
#     print(num, prq, pts)

In [312]:
# make sure elements of each list look as expected
# numbers
# prereqs
# points

Again, I save each row in a list to append to a list of lists, which will be the data for the DataFrame.

In [330]:
catalog_data = []

for row in rows:
    spans = row.select('p.bold')
    if spans == []:
        continue
    row_data = []

    m1 = re.search("C?S?C?I?M?A?I?N?T?C?O?R?E?F?R?S?E?M?\-U?G?A\.\d{4}", spans[0].text.strip())
    if m1!=None:
        num = m1[0]
    row_data.append(num)
       
    m2 = re.search("Prerequisites: (.*)", spans[2].text.strip())
    if m2!=None:
        prq = m2[1]
    row_data.append(prq)
    
    m3 = re.search("(.*) Points", spans[1].text.strip())
    if m3!=None:
        pts = m3[1]
    else:
        pts = None
    row_data.append(pts)
    
#     print(row_data)
    catalog_data.append(row_data)

# check data list
# catalog_data

I create my DataFrame with the list of lists created above and a list of column names.

In [332]:
# create dataframe
cols = ['Number', 'Prereqs', 'Points']
df2 = pd.DataFrame(catalog_data, columns=cols)
# pd.set_option('display.max_rows', 200)
pd.set_option('display.max_rows', 10)
df2

,Number,Prereqs,Points
0,CSCI-GA.1133,None,4
1,CSCI-GA.1144,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA.1170,At least one year of experience with a high-le...,3
3,CSCI-GA.1180,None,3
4,CSCI-GA.2110,Students taking this class should already have...,3
...,...,...,...
95,CSCI-UA.0897,Restricted to declared computer science majors...,1 - 4
96,CSCI-UA.0898,Restricted to declared computer science majors...,1 - 4
97,CSCI-UA.0997,Permission of the department. Does not satisfy...,1 - 4
98,CSCI-UA.0998,Permission of the department. Does not satisfy...,1 - 4


### Show results

In [333]:
# info to show data types and counts
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Number   100 non-null    object
 1   Prereqs  100 non-null    object
 2   Points   99 non-null     object
dtypes: object(3)
memory usage: 2.5+ KB


In [334]:
# first 5 rows
df2.head()

,Number,Prereqs,Points
0,CSCI-GA.1133,None,4
1,CSCI-GA.1144,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA.1170,At least one year of experience with a high-le...,3
3,CSCI-GA.1180,None,3
4,CSCI-GA.2110,Students taking this class should already have...,3


In [335]:
# last 5 rows
df2.tail()

,Number,Prereqs,Points
95,CSCI-UA.0897,Restricted to declared computer science majors...,1 - 4
96,CSCI-UA.0898,Restricted to declared computer science majors...,1 - 4
97,CSCI-UA.0997,Permission of the department. Does not satisfy...,1 - 4
98,CSCI-UA.0998,Permission of the department. Does not satisfy...,1 - 4
99,FRSEM-UA.0597,"Some programming experience in Python, Java, J...",4


In [336]:
# random sampling of 5 rows
df2.sample(5)

,Number,Prereqs,Points
84,CSCI-UA.0472,Computer Systems Organization (CSCI-UA 201) an...,4
70,CSCI-UA.0102,Introduction to Computer Science (CSCI-UA 101)...,4
83,CSCI-UA.0470,Computer Systems Organization (CSCI-UA 201).,4
8,CSCI-GA.2246,An understanding of modern operating systems a...,3
57,CSCI-GA.3813,Permission of the faculty project supervisor a...,"1-3 (MS), 1-12 (PhD)"


## 3. Put together both DataFrames

I merge the two DFs from above on the 'Number' column with the how='left' parameter.

In [337]:
bigdf = pd.merge(df, df2, on="Number", how="left")
bigdf

,Number-Section,Name,Instructor,Time,Number,Section,Prereqs,Points
0,CSCI-GA.1144-001,PAC II*,Mohamed Zahran,T 6:00-8:30PM,CSCI-GA.1144,001,CSCI-GA 1133 or departmental permission.,4
1,CSCI-GA.1144-002,PAC II Recitation,Gurkirat Singh Bajwa,R 7:10-8:00PM,CSCI-GA.1144,002,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA.1170-001,Fundamental Algorithms,Yevgeniy Dodis,T 7:10-9:00PM,CSCI-GA.1170,001,At least one year of experience with a high-le...,3
3,CSCI-GA.1170-002,Fundamental Algorithms Recitation,"Alex Bienstock, Charles Peyser, Fengyuan Liu",R 8:10-9:00PM,CSCI-GA.1170,002,At least one year of experience with a high-le...,3
4,CSCI-GA.1170-003,Fundamental Algorithms Recitation,Harish Karthikeyan,R 8:10-9:00PM,CSCI-GA.1170,003,At least one year of experience with a high-le...,3
...,...,...,...,...,...,...,...,...
145,CSCI-UA.0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 9:30-10:45AM,CSCI-UA.0480,057,Topics determine prerequisites.,4
146,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,MW 3:30-4:45PM,CSCI-UA.0480,069,Topics determine prerequisites.,4
147,CSCI-UA.0480-521,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 11:00-12:15PM,CSCI-UA.0480,521,Topics determine prerequisites.,4
148,CSCI-UA.0480-522,Special Topics: Algorithmic Problem Solving,Kunal Khatri,W 3:30-4:45PM,CSCI-UA.0480,522,Topics determine prerequisites.,4


I create a final DF that only shows the desired columns in the desired order, using the columns from the merged DF above.

In [340]:
finaldf = bigdf[['Number', 'Name', 'Instructor', 'Time', 'Prereqs', 'Points']]
# pd.set_option('display.max_rows', 200)
pd.set_option('display.max_rows', 10)
finaldf

,Number,Name,Instructor,Time,Prereqs,Points
0,CSCI-GA.1144,PAC II*,Mohamed Zahran,T 6:00-8:30PM,CSCI-GA 1133 or departmental permission.,4
1,CSCI-GA.1144,PAC II Recitation,Gurkirat Singh Bajwa,R 7:10-8:00PM,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA.1170,Fundamental Algorithms,Yevgeniy Dodis,T 7:10-9:00PM,At least one year of experience with a high-le...,3
3,CSCI-GA.1170,Fundamental Algorithms Recitation,"Alex Bienstock, Charles Peyser, Fengyuan Liu",R 8:10-9:00PM,At least one year of experience with a high-le...,3
4,CSCI-GA.1170,Fundamental Algorithms Recitation,Harish Karthikeyan,R 8:10-9:00PM,At least one year of experience with a high-le...,3
...,...,...,...,...,...,...
145,CSCI-UA.0480,Special Topics: Natural Language Processing,Adam Meyers,TR 9:30-10:45AM,Topics determine prerequisites.,4
146,CSCI-UA.0480,Special Topics: Agile Software Development and...,Amos Bloomberg,MW 3:30-4:45PM,Topics determine prerequisites.,4
147,CSCI-UA.0480,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 11:00-12:15PM,Topics determine prerequisites.,4
148,CSCI-UA.0480,Special Topics: Algorithmic Problem Solving,Kunal Khatri,W 3:30-4:45PM,Topics determine prerequisites.,4


## 4. Conclusion

**Anomalies, Discrepancies, Unexpected Data/Relationships Between Data & Potential Fixes**

_In the Spring 2021 Course Schedule..._
* the course number-section had a strange (whitespace?) character in it that I had to remove for the regex to work properly and for the number-section to display as I wanted.
* I expected only CSCI-UA or CSCI-GA courses, but there were MAINT-GA courses that had to be accounted for in the regex I used.
* some courses were cross-listed in other departments (such as Data Science or Math), so I disregarded these non-CS numbers and only used the CS numbers (which were always the first numbers listed).
* there were many anomalies in the Instructor data ("Office Hours", newline and whitespace characters that couldn't be seen when printed out, classes labeled as canceled where the instructor should be...) that were dealt with through regex and match object comparison before creating the dataframe. For courses with multiple instructors, I placed a comma and space between the instructors' names.
* I chose to keep all rows, including those for recitation sections (this is relevant to combining the dataframes).

_In the Course Catalog..._
* the same issue with MAINT-GA as above (as well as FRSEM-UA) occurred here.
* there were different formats for points (one specific point value, a range of points, ranges of points for different levels of study, etc.), so I retained all of this information as a string (object) rather than choosing one point value for each course to turn the points column int type int. There was one course with no point value listed, so I turned this into missing data rather than assuming the point value.

_Combining into one DataFrame:_
* Recitation sections don't have a point value, but since I matched rows between the two dataframes based on course number, the point value for the course itself was also applied to its recitations. I chose to leave this as is, but the points for recitations could be changed to zero by changing the points column to '0' if 'Recitation' is contained in the name column.

**About the Resulting DataFrame:**

Using the how='left' parameter in the pd.merge function made it so the resulting dataframe kept all the rows from the first dataframe, which was the course schedule dataframe. This is important, as the first (course schedule) dataframe is larger than the second (course catalog) dataframe. For courses in the course schedule with numbers that appeared in the course catalog, their prereqs and points columns were filled in with the appropriate values. However, if a course in the course schedule did not appear in the course catalog (e.g. CSCI-UA.0074	- Big Ideas: Artificial Intelligence), the prereqs and points columns were left with missing values.